In [1]:
import numpy as np
import sys
import scipy.stats as st
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import combinations
sys.path.append('../multiKS/')
from multiKS import multiKS
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import pickle

In [2]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [36]:
dim = 4
mu = [0]*dim
sigma = np.identity(dim)

#mu = [1,2,3,0]
#sigma = [[1,0,0,0], [0, 1, 0.1, 0.5], [0.2, 1, 2, 1], [0, 0.5, 0, 1]]

rv = st.multivariate_normal(mu, sigma)
def KS_wrapper(Xs):
    return [multiKS(X, rv.cdf) for X in Xs]

In [40]:
mc_loops_per_core = 40
n_cores = 8
results = {}

#for N in [10, 20, 30, 40, 50, 75, 100, 150, 200, 300]:
for N in [50]:
    print(f'N={N}')
    samples = [[rv.rvs(N) for i in range(mc_loops_per_core)] for j in range(n_cores)]
    with ProcessPoolExecutor() as executor:
        res = executor.map(KS_wrapper, samples)
    results[N] = flatten(res)
    #with open(f'KS_results_dim={dim}.pickle', 'wb') as handle:
    #    pickle.dump(results, handle)

N=50


In [35]:
np.quantile(results[30], 0.9), np.quantile(results[30], 0.95)

(0.9309422655249323, 0.9509319447934322)

In [39]:
np.quantile(results[30], 0.8), np.quantile(results[30], 0.9), np.quantile(results[30], 0.95)

(0.8751609284241965, 0.909591013723301, 0.9370494481824921)